## Parameterize MLP 

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import pandas as pd

def mlpResult(structure):

    fishModel = Sequential()
    for index in range(len(structure)):
       if index == 0:
        fishModel.add(Dense(structure[index], input_shape = (27,), activation = "relu"))
        continue
       
       fishModel.add(Dense(structure[index], activation = "relu"))
        
     
    fishModel.add(Dense(1, activation = "linear"))
    fishModel.compile(optimizer = Adam(), loss = "mean_squared_error", metrics = ["mean_squared_error"])
    print(fishModel.summary())
    return fishModel

## Parameterize KAN

In [2]:
import torch
from kan import *
from kan.utils import create_dataset_from_data

def kanResult(Structure, GridSize):
    return KAN(width = Structure, grid = GridSize, seed = 42)


## Compare the Performance of KAN versus MLP

In [10]:
import numpy as np
from sklearn.metrics import mean_squared_error
import torch
from kan import *
from kan.utils import create_dataset_from_data


modType = []
Params = []
Layers = []
GridSize = []
Error = []

Data = pd.read_csv("mlpData.csv").drop( ["Date", "Unnamed: 0"],axis = 1)

trainData = Data[Data["Year"] < 2021]

testData = Data[Data["Year"] > 2020]

xTestData = testData.drop(["Target"], axis = 1)
yTestData = testData["Target"]
    
xMLP = trainData.drop(["Target"], axis = 1)
yMLP = trainData["Target"]

xKAN = torch.tensor(xMLP.values, dtype=torch.float32)
yKAN = torch.tensor(yMLP.values, dtype=torch.float32)
yKAN = yKAN.reshape(-1,1)

data = create_dataset_from_data(xKAN, yKAN, train_ratio = 1)

for sample in range(0,300,1):

    np.random.seed(sample)

    hidden = round(np.random.normal(loc = 3, scale = 1))
    if hidden <= 0:
        hidden = hidden * -1 + 1
        
    mlpStructure = []
    kanStructure = []
    
    for i in range(0,hidden,1):
        
        nuerons = (round(np.random.normal(loc = 25, scale = 10)))
        kanNuerons = (round(np.random.normal(loc = 4, scale = 1)))
        if nuerons <= 0:
            nuerons = nuerons * -1 + 1
            
        mlpStructure.append(nuerons)
        kanStructure.append(kanNuerons)

    kanStructure.append(1)
    kanStructure[0] = 27
        
    
    gridSize = round(np.random.normal(loc = 9, scale = 3))
    if gridSize <= 0:
        gridSize = gridSize * -1 +1

    mlpParams = 0
    for index in range(len(mlpStructure)):
        if index == 0:
            mlpParams += mlpStructure[index] * 27 + mlpStructure[index]
        else:
            mlpParams += mlpStructure[index] * mlpStructure[index-1] + mlpStructure[index]
    mlpParams += mlpStructure[-1] + 1

    MLP = mlpResult(mlpStructure)
    MLP.fit(xMLP,yMLP, epochs = 10, batch_size = 1, validation_split = 0)
    Predictions = MLP.predict(xTestData)
    Eps = mean_squared_error(yTestData, Predictions)
    
    Error.append(Eps)
    Params.append(mlpParams)
    modType.append("MLP")
    Layers.append(len(mlpStructure))
    GridSize.append(None)
    
    kanParams = 0

    for index in range(len(kanStructure)):
        if index == 0:
            kanParams += kanStructure[index]
        else:
            kanParams += kanStructure[index] * kanStructure[index-1] 

    kanParams = kanParams * 3 * gridSize

    print(kanStructure)
    KAN =  KAN(width = kanStructure, grid = gridSize, seed = sample)
    
    KAN.fit(data, opt="LBFGS", steps=50, lamb=0.001, loss_fn = torch.nn.MSELoss())
    Predictions = KAN(torch.tensor(xTestData.values, dtype=torch.float32))

    if torch.isnan(Predictions).any() == True:
        print("ERROR ON SAMPLE", sample)
        continue 
        
    Eps = mean_squared_error(yTestData, Predictions.detach().numpy())

    Error.append(Eps)
    Params.append(kanParams)
    modType.append("KAN")
    Layers.append(len(kanStructure)-1)
    GridSize.append(gridSize)
    
    
    
    
Results = pd.DataFrame({                    
                        "modType": modType,
                        "Params": Params,
                        "Hidden_Layers": Layers,
                        "GridSize": GridSize,
                        "Error": Error
                      })

Results.to_csv("Results.csv") 

Model: "sequential_59"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_276 (Dense)           (None, 29)                812       
                                                                 
 dense_277 (Dense)           (None, 47)                1410      
                                                                 
 dense_278 (Dense)           (None, 15)                720       
                                                                 
 dense_279 (Dense)           (None, 23)                368       
                                                                 
 dense_280 (Dense)           (None, 29)                696       
                                                                 
 dense_281 (Dense)           (None, 1)                 30        
                                                                 
Total params: 4,036
Trainable params: 4,036
Non-train

description:   0%|                                                           | 0/50 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/kan/MultKAN.py:798: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1760.)
  self.subnode_actscale.append(torch.std(x, dim=0).detach())
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/kan/MultKAN.py:808: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1760.)
  input_range = torch.std(preacts, dim=0) + 0.1
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-pack

saving model version 0.1
Model: "sequential_60"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_282 (Dense)           (None, 19)                532       
                                                                 
 dense_283 (Dense)           (None, 14)                280       
                                                                 
 dense_284 (Dense)           (None, 2)                 30        
                                                                 
 dense_285 (Dense)           (None, 17)                51        
                                                                 
 dense_286 (Dense)           (None, 23)                414       
                                                                 
 dense_287 (Dense)           (None, 1)                 24        
                                                                 
Total params: 1,331
Trainabl

None
Epoch 1/10
931/931 [==============================] - 2s 2ms/step - loss: 38149348.0000 - mean_squared_error: 38149348.0000
Epoch 2/10
931/931 [==============================] - 2s 2ms/step - loss: 24762404.0000 - mean_squared_error: 24762404.0000
Epoch 3/10
931/931 [==============================] - 2s 2ms/step - loss: 22115172.0000 - mean_squared_error: 22115172.0000
Epoch 4/10
931/931 [==============================] - 2s 2ms/step - loss: 20840048.0000 - mean_squared_error: 20840048.0000
Epoch 5/10
931/931 [==============================] - 2s 2ms/step - loss: 21951914.0000 - mean_squared_error: 21951914.0000
Epoch 6/10
931/931 [==============================] - 2s 2ms/step - loss: 20251892.0000 - mean_squared_error: 20251892.0000
Epoch 7/10
931/931 [==============================] - 2s 2ms/step - loss: 19579716.0000 - mean_squared_error: 19579716.0000
Epoch 8/10
931/931 [==============================] - 2s 2ms/step - loss: 18830662.0000 - mean_squared_error: 18830662.0000
Epo

description:   0%|                                                           | 0/50 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/kan/MultKAN.py:798: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1760.)
  self.subnode_actscale.append(torch.std(x, dim=0).detach())
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/kan/MultKAN.py:808: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1760.)
  input_range = torch.std(preacts, dim=0) + 0.1
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-pack

...

description:   0%|                                                           | 0/50 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/kan/MultKAN.py:798: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1760.)
  self.subnode_actscale.append(torch.std(x, dim=0).detach())
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/kan/MultKAN.py:808: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1760.)
  input_range = torch.std(preacts, dim=0) + 0.1
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-pack

saving model version 0.1
ERROR ON SAMPLE 298
Model: "sequential_358"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1485 (Dense)          (None, 8)                 224       
                                                                 
 dense_1486 (Dense)          (None, 25)                225       
                                                                 
 dense_1487 (Dense)          (None, 1)                 26        
                                                                 
Total params: 475
Trainable params: 475
Non-trainable params: 0
_________________________________________________________________


None
Epoch 1/10
931/931 [==============================] - 2s 2ms/step - loss: 40084132.0000 - mean_squared_error: 40084132.0000
Epoch 2/10
931/931 [==============================] - 1s 2ms/step - loss: 26135110.0000 - mean_squared_error: 26135110.0000
Epoch 3/10
931/931 [==============================] - 1s 1ms/step - loss: 23802340.0000 - mean_squared_error: 23802340.0000
Epoch 4/10
931/931 [==============================] - 1s 2ms/step - loss: 22716684.0000 - mean_squared_error: 22716684.0000
Epoch 5/10
931/931 [==============================] - 3s 3ms/step - loss: 21674016.0000 - mean_squared_error: 21674016.0000
Epoch 6/10
931/931 [==============================] - 2s 3ms/step - loss: 21323158.0000 - mean_squared_error: 21323158.0000
Epoch 7/10
931/931 [==============================] - 2s 2ms/step - loss: 21011242.0000 - mean_squared_error: 21011242.0000
Epoch 8/10
931/931 [==============================] - 1s 2ms/step - loss: 19946992.0000 - mean_squared_error: 19946992.0000
Epo

description:   0%|                                                           | 0/50 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/kan/MultKAN.py:798: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1760.)
  self.subnode_actscale.append(torch.std(x, dim=0).detach())
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/kan/MultKAN.py:808: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1760.)
  input_range = torch.std(preacts, dim=0) + 0.1
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-pack

saving model version 0.1
